In [1]:
import torch as tch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.distributions.uniform import Uniform
import pytorch_lightning as pl
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
device = 'cuda' if tch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
pi = tch.tensor(math.pi).float().to(device)
pi

tensor(3.1416)

In [4]:
class RotationDataset(Dataset):
    def __init__(self, size):
        # Generate Positive Datasets
        unif1 = Uniform(0.1, 1)
        unif2 = Uniform(0, 2*math.pi)
        unif3 = Uniform(0, 2*math.pi)
        r = unif1.sample([size]).float()
        th = unif2.sample([size]).float()
        self.X = tch.column_stack([r*tch.cos(th), r*tch.sin(th)])
        thetas = unif3.sample([size]).float()
        X_p = tch.empty((size, 2))
        for i, (x, th) in enumerate(zip(self.X, thetas)):
            R = tch.tensor([[tch.cos(th), tch.sin(th)], [-tch.sin(th), tch.cos(th)]]).float()
            x_p = tch.mm(x.view(1,2), R)
            X_p[i] = x_p
        
        # Generate Negative Datasets
        r_perm = tch.randperm(X_p.shape[0])
        X_n = tch.column_stack([X_p[:,0], X_p[r_perm, 1]])

        # Stack Pairs
        self.X_pairs = tch.vstack([X_p, X_n])
        self.X = tch.vstack([self.X, self.X])

        # Labeling
        y_p = tch.zeros((X_p.shape[0], 1))
        y_n = tch.ones((X_n.shape[0], 1))
        self.y = tch.vstack([y_p, y_n])

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        x = tch.Tensor(self.X[idx])
        x_pair = tch.Tensor(self.X_pairs[idx])
        y = tch.Tensor(self.y[idx])
        return x, x_pair, y

In [5]:
class ContrastiveLoss(tch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = tch.mean((1-label) * tch.pow(euclidean_distance, 2) +
                                      (label) * tch.pow(tch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

In [6]:
class SNN(pl.LightningModule):
    def __init__(self, margin, lr, batch_size, train_size, val_size, test_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 100),
            nn.ReLU(),
            nn.Linear(100, 1),
            nn.ReLU()
        )
        self.criterion = ContrastiveLoss(margin)
        self.lr = lr
        self.batch_size = batch_size
        self.train_size = train_size
        self.val_size = val_size
        self.test_size = test_size
        self.dist = margin * 0.3

    def forward(self, x1, x2):
        out1 = self.model(x1)
        out2 = self.model(x2)
        return out1, out2
    
    def acc(self, out1, out2, y):
        correct = tch.count_nonzero((tch.abs(out1 - out2) <= self.dist) != y).item()
        correct /= self.batch_size
        return correct
    
    def training_step(self, batch, batch_idx):
        x, x_p, y = batch
        out1, out2 = self(x, x_p)
        loss = self.criterion(out1, out2, y)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, x_p, y = batch
        out1, out2 = self(x, x_p)
        loss = self.criterion(out1, out2, y)
        acc = self.acc(out1, out2, y)
        
        metrics = {"val_loss": loss, "val_acc": acc}
        
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, x_p, y = batch
        out1, out2 = self(x, x_p)
        loss = self.criterion(out1, out2, y)
        acc = self.acc(out1, out2, y)
        
        metrics = {"test_loss": loss, "test_acc": acc}
        
        self.log_dict(metrics)
        return loss
    
    def configure_optimizers(self):
        optimizer = tch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer
    
    def train_dataloader(self):
        return DataLoader(RotationDataset(self.train_size), batch_size=self.batch_size, shuffle=True, num_workers=16)
    
    def val_dataloader(self):
        return DataLoader(RotationDataset(self.val_size), batch_size=self.batch_size, num_workers=16)
    
    def test_dataloader(self):
        return DataLoader(RotationDataset(self.test_size), batch_size=self.batch_size, num_workers=16)

In [7]:
model = SNN(margin=2, lr=1e-3, batch_size=500, train_size=100000, val_size=10000, test_size=10000)

In [8]:
model

SNN(
  (model): Sequential(
    (0): Linear(in_features=2, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=1, bias=True)
    (3): ReLU()
  )
  (criterion): ContrastiveLoss()
)

In [9]:
trainer = pl.Trainer(max_epochs=500, progress_bar_refresh_rate=20)
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type            | Params
----------------------------------------------
0 | model     | Sequential      | 401   
1 | criterion | ContrastiveLoss | 0     
----------------------------------------------
401       Trainable params
0         Non-trainable params
401       Total params
0.002     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [10]:
trainer.test()

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9907000064849854, 'test_loss': 0.04323258996009827}
--------------------------------------------------------------------------------


[{'test_loss': 0.04323258996009827, 'test_acc': 0.9907000064849854}]